# Wrapping a subset of a very large library

In this example, we aim at presenting the methodology to wrap only a subset of a library.
For the purpose of illustration and to demonstrate the usage of **AutoWIG** for very complex libraries, we considered the wrapping of the **Clang** library, a complete *C*/*C++* compiler.
**Clang** is a great tool, but its stable *Python* interface (i.e. **libclang**) is lacking some useful features that are needed by **AutoWIG**.
In particular, class template specializations are not available in the abstract syntax tree.
Fortunately, most of the classes that would be needed during the traversal of the *C++* abstract syntax tree are not template specializations.
We therefore proposed to bootstrap the **Clang** *Python* bindings with a first version of **AutoWIG** that uses only **libclang**.
The new **Clang** *Python* interface, produced by **AutoWIG**, is **PyClangLite**.
**PyClangLite** is able to wrap class template specializations.
As for **libclang**, this interface is proposed only for a subset of the **Clang** library sufficient enough for proposing the new `pyclanglite` `parser`.

First, ensure that the **LLVM**/**Clang** technologies are installed on your computer and if there were built with `RTTI`.

In [1]:
import subprocess
if not subprocess.check_output(["llvm-config", "--version"]).strip() == '3.8.1':
    raise Exception
subprocess.check_call(['clang++', '--version'])
if not subprocess.check_output(["llvm-config", "--has-rtti"]).strip() == 'YES':
    raise Exception

Note that these techonologies must be installed using the current system prefix

In [2]:
import sys
from path import path
prefix = path(subprocess.check_output(["llvm-config", "--prefix"]).strip())
if not prefix == sys.prefix:
    raise Exception

Then, clone the **PyClangLite**  repository into the **PyClangLite** directory.

In [3]:
from path import path
srcdir = path('PyClangLite')
if not srcdir.exists():
    from git import Repo
    repo = Repo.clone_from('https://github.com/StatisKit/PyClangLite.git', srcdir.relpath('.'))

This repository already has wrappers, we therefore need to remove them.

In [4]:
srcdir = srcdir/'src'/'py'
for wrapper in srcdir.walkfiles('*.cpp'):
    wrapper.unlink()

In addition to the **Clang** libraries, the **ClangLite** library is needed in order to have access to some functionalities.

In [5]:
from autowig.scons import scons
if not prefix.basename() == '_build':
    print scons(srcdir.parent.parent, 'cpp', '--prefix=' + prefix, '-j6')

scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "src/cpp/tool.h" as "/home/pfernique/.miniconda2/envs/autowig/include/clanglite/tool.h"
g++ -o src/cpp/tool.os -c -std=c++0x -fvisibility-inlines-hidden -ffunction-sections -fdata-sections -Wno-deprecated-declarations -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS -I/home/pfernique/.miniconda2/envs/autowig/include -I/usr/include/python2.7 src/cpp/tool.cpp
scons: done building targets.



The `tool.h` header of this **ClangLite** library includes all necessary **Clang** headers.

In [6]:
headers = [prefix/'include'/'clanglite'/'tool.h']

We import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).


In [7]:
from autowig import autowig
asg = autowig.AbstractSemanticGraph()

We then parse the header with relevant compilation flags.

In [8]:
autowig.parser.plugin = 'libclang'
asg = autowig.parser(asg, headers,
               flags = ['-x', 'c++', '-std=c++11',
                        '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS',
                        '-I' + str((prefix/'include').abspath())],
               libpath = prefix/'lib'/'libclang.so',
               bootstrap = False,
               silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation could be suitable.
Nevertheless, we need to force some *C++* components to be wrapped or not.
Moreover, **libclang** does not allow to investigate if enumerations are scoped or not.
We therefore implements a new `controller`.

In [17]:
def clanglite_controller(asg):
    
    for node in asg['::boost::python'].classes(nested = True):
        node.is_copyable = True
        
    for node in asg.classes():
        node.boost_python_export = False
    for node in asg.enumerations():
        node.boost_python_export = False
    for node in asg.enumerators():
        if node.parent.boost_python_export:
            node.boost_python_export = False
            
    for node in asg.functions(free = True):
        node.boost_python_export = False
    for node in asg.variables(free = True):
        node.boost_python_export = False
        
    from autowig.default_controller import refactoring
    asg = refactoring(asg)
    for fct in asg['::clanglite'].functions():
        if not fct.localname == 'build_ast_from_code_with_args':
            fct.parent = fct.parameters[0].qualified_type.desugared_type.unqualified_type
    
    subset = []
    classes = [asg['class ::clang::Type'], asg['class ::clang::Decl']]
    subset += classes
    subset += classes[0].subclasses(recursive=True)
    subset += classes[1].subclasses(recursive=True)
    subset.append(asg['class ::llvm::StringRef'])
    subset.append(asg['class ::clang::ASTUnit'])
    subset.append(asg['class ::clang::ASTContext'])
    subset.append(asg['class ::clang::FileID'])
    subset.append(asg['class ::clang::SourceLocation'])
    subset.append(asg['class ::clang::CXXBaseSpecifier'])
    subset.append(asg['class ::clang::DeclContext'])
    subset.append(asg['enum ::clang::AccessSpecifier'])
    subset.append(asg['enum ::clang::LinkageSpecDecl::LanguageIDs'])
    subset.append(asg['enum ::clang::BuiltinType::Kind'])
    subset.append(asg['enum ::clang::TemplateArgument::ArgKind'])
    subset.append(asg['enum ::clang::Decl::Kind'])
    subset.extend(asg['::boost::python'].classes(nested = True))
    subset.extend(asg['::boost::python'].enumerations(nested = True))
    subset.extend(asg.nodes('::clanglite::build_ast_from_code_with_args'))

    for node in subset:
        node.boost_python_export = True

    if autowig.parser.plugin == 'libclang':
        for node in (asg.functions(pattern='.*(llvm|clang).*_(begin|end)')
                     + asg.functions(pattern='.*(llvm|clang).*getNameAsString')
                     + asg.nodes('::clang::NamedDecl::getQualifiedNameAsString')
                     + asg.nodes('::clang::ObjCProtocolDecl::collectInheritedProtocolProperties')
                     + asg.nodes('::clang::ASTUnit::LoadFromASTFile')
                     + asg.nodes('::clang::ASTUnit::getCachedCompletionTypes')
                     + asg.nodes('::clang::ASTUnit::getBufferForFile')
                     + asg.nodes('::clang::CXXRecordDecl::getCaptureFields')
                     + asg.nodes('::clang::ASTContext::SectionInfos')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::ASTContext::getObjCEncodingForFunctionDecl')
                     + asg.nodes('::clang::ASTContext::getObjCEncodingForPropertyDecl')
                     + asg.nodes('::clang::ASTContext::getObjCEncodingForMethodDecl')
                     + asg.nodes('::clang::ASTContext::getAllocator')):
            node.boost_python_export = False
            
    import sys
    from path import path
    for header in (path(sys.prefix)/'include'/'clang').walkfiles('*.h'):
        asg[header.abspath()].is_external_dependency = False
    
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [10]:
autowig.controller['clanglite'] = clanglite_controller
autowig.controller.plugin = 'clanglite'
asg = autowig.controller(asg)

In order to wrap a subset of the **Clang** library, the user need to implements a `generator` that passes particular nodes to the `boost_python` `generator`.

In [11]:
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                  module = srcdir/'_clanglite.cpp',
                  decorator = srcdir/'clanglite'/'_clanglite.py',
                  closure = False)

The wrappers are only generated in-memory.
We therefore need to write them on the disk to complete the process.

In [12]:
for wrapper in wrappers:
    wrapper.write()